In [ ]:
# this is Chang Liu's code for week 6 practices.
import warnings

# Temporarily suppress runtime warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

# 1. Prob of hidden path
# file_path = "../../downloads/dataset_35946_8.txt"
# with open (file_path,'r') as file:
#     lines = file.readlines()

# seq = lines[0].strip()
# state = lines[2].strip().split()
# matrix_lines = lines[4:]
# probMatrix = []
# for line in matrix_lines:
#      # Splitting each line by tabs and filtering out empty strings or non-numeric values
#     row = [float(value) for value in line.strip().split('\t') if value.replace('.', '', 1).isdigit()]
#     if row:
#         probMatrix.append(row)

def HiddenPathProb(path, state, probMatrix):
    # assume that any initial state is 0.5
    probability = 0.5
    for i in range(len(path)-1):
        index1 = state.index(path[i])
        index2 = state.index(path[i+1])
        probability *= probMatrix[index1][index2]

    return probability


# 2. probOutcome
# file_path = "../../downloads/dataset_35946_10.txt"
# with open (file_path,'r') as file:
#     lines = file.readlines()

# outcome = lines[0].strip()
# signal = lines[2].strip().split()
# hiddenPath = lines[4].strip()
# state = lines[6].strip().split()
# matrix_lines = lines[8:]
# emissionMatrix = []
# for line in matrix_lines:
#      # Splitting each line by tabs and filtering out empty strings or non-numeric values
#     row = [float(value) for value in line.strip().split('\t') if value.replace('.', '', 1).isdigit()]
#     if row:
#         emissionMatrix.append(row)

def probOutcome(outcome, signal, hiddenpath, state, emission):
    # use i to traverse both hiddenPath and outcome sequence
    # emission is a state * signal matrix
    prob = 1
    for i in range(len(hiddenpath)):
        stateIndex = state.index(hiddenpath[i])
        signalIndex = signal.index(outcome[i])
        prob *= emission[stateIndex][signalIndex]

    return prob


# 3. viterbi algorithm
# file_path = "../../downloads/dataset_35947_7.txt"
# with open (file_path,'r') as file:
#     lines = file.readlines()

# outcome = lines[0].strip()
# signal = lines[2].strip().split()
# # we are finding the optimal path and no hidden path is given
# #hiddenPath = lines[4].strip()
# state = lines[4].strip().split()
# matrix_lines1 = lines[6:10]
# transitionMatrix = []
# for line in matrix_lines1:
#      # Splitting each line by tabs and filtering out empty strings or non-numeric values
#     row = [float(value) for value in line.strip().split('\t') if value.replace('.', '', 1).isdigit()]
#     if row:
#         transitionMatrix.append(row)

# # typically it starts from line 10, but in the case of integer outcome, lines should adjust to start from 12
# matrix_lines2 = lines[10:]
# emissionMatrix = []
# for line in matrix_lines2:
#      # Splitting each line by tabs and filtering out empty strings or non-numeric values
#     row = [float(value) for value in line.strip().split('\t') if value.replace('.', '', 1).isdigit()]
#     if row:
#         emissionMatrix.append(row)

# print(transitionMatrix)
# print(emissionMatrix)
def viterbi(outcome, signal, state, transitionMatrix, emissionMatrix):
    hiddenPath = ''
    
    # score is a matrix state*len(outcome), exclude the source point
    # score is the best score at each position of outcome, based on each state
    score = [[0 for _ in range(len(outcome))] for _ in range(len(state)) ]
    stateTrack = [[float('-inf') for _ in range(len(outcome))] for _ in range(len(state)) ]
    # base case, the initial probability of each state is equal
    signalIndex = signal.index(outcome[0])
    # for i in range(len(state)):
    #     score[i][0] = 1 * (1/len(state)) * emissionMatrix[i][signalIndex]
    score[0][0] = 1
    # traverse the rest of outcome
    for i in range(1, len(outcome)):
        # the current node is s_j,i
        for j in range(len(state)):
            # the current signal
            signalIndex = signal.index(outcome[i])
            # n is the possible previous state
            max_value, max_n = max(
                ((score[n][i-1]*transitionMatrix[n][j]*emissionMatrix[j][signalIndex], n) for n in range(len(state))),
                key=lambda x: x[0]
                )
            score[j][i] = max_value
            stateTrack[j][i] = max_n
    
    i = len(outcome)-1
    # find the largest prob in last column and backtrack the seq
    
    _, max_n = max(((score[n][i],n) for n in range(len(state))), key=lambda x: x[0])
    hiddenPath = hiddenPath + state[max_n]
    
    while i > 0:
        max_n = stateTrack[max_n][i]
        hiddenPath = state[max_n] + hiddenPath
        i -= 1
    
        
    return hiddenPath


# 4. Profile HMM
file_path = "../../downloads/dataset_35949_5.txt"
with open(file_path,"r") as file:
    lines = file.readlines()

threshold = 0.239
pseudocount = 0.01
alphabet = lines[2].strip().split()
alignments_lines = lines[4:]
alignments =[] # list of strings
for line in alignments_lines:
    line = line.strip()
    if line:
        alignments.append(line.strip())

import numpy as np
import pandas as pd
def profileHMM_para(threshold, alphabet, alignments):
    # find the alignment_star accoridng to threshold
    align_size = len(alignments)
    string_size = len(alignments[0])
    # alignment_star = alignments.copy()
    columns_to_remove = set()
    for i in range(string_size):
        fraction_count = 0
        for j in range(align_size):
            if alignments[j][i] == '-':
                fraction_count += 1
        ratio = fraction_count/align_size
        
        if ratio >= threshold:
            columns_to_remove.add(i)

    # alignment_star stores the columns index that will be considered for the profile matrix.
    col_index = [i for i in range(len(alignments[0]))]
    alignments_star = [item for item in col_index if item not in columns_to_remove]
    
    # seed alignment defines the length of insertion states, match states, and deletion states 
    # generate and store all possible states nodes
    states = ["S", "I0"]
    for i in range(1, len(alignments_star)+1):
        states.append("M"+str(i))
        states.append("D"+str(i))
        states.append("I"+str(i))
    states.append("E")

    dimension = len(states)
    # build a transmissionMatrix, it is initialized as a count matrix, and will calculate into transmission probability later
    transmissionMatrix = [[0 for _ in range(dimension)]for _ in range(dimension)]
    emissionMatrix = [[0 for _ in range(len(alphabet))]for _ in range(dimension)]

    # traverse every string in alignments
    all_statePath = []
    for i in range(len(alignments)):
        statePath = []
        for j in range(len(alignments[i])):
            # insertion case
            if j in columns_to_remove:
                if alignments[i][j] != "-":
                    statePath.append("I")
                # here i should be j-1 state how to know?
                #transmissionMatrix[i][insertion] += 1
            # match case
            else:
                if alignments[i][j] != "-":
                    statePath.append("M")
                # deletion case
                elif alignments[i][j] == "-":
                    statePath.append("D")
        all_statePath.append(statePath)
        
    for s in range(len(all_statePath)):
        # deal with the start point
        currentCol = 1
        statePath = all_statePath[s]
        intervals = list(columns_to_remove.copy())
        if statePath[0] == "M":
            transmissionMatrix[0][states.index("M1")] += 1
            emissionMatrix[states.index("M1")][alphabet.index(alignments[s][alignments_star[0]])] += 1
        elif statePath[0] == "D":
            transmissionMatrix[0][states.index("D1")] += 1
        else:
            transmissionMatrix[0][states.index("I0")] += 1
            if alignments[s][intervals[0]] != "-":
                emissionMatrix[states.index("I0")][alphabet.index(alignments[s][intervals[0]])] += 1
                intervals.remove(intervals[0])
        
        
        for i in range(1, len(statePath)):
            # dealing with the start point is an example of this operation
            lastState = statePath[i-1]+ (str(currentCol) if statePath[i-1]!="I" else str(currentCol-1))
            currentCol = int(lastState[-1])
            currentState = statePath[i]+ (str(currentCol+1) if statePath[i]!="I" else str(currentCol))
            
            if statePath[i] == "M":
                star_index = int(currentState[-1])-1
                emissionMatrix[states.index(currentState)][alphabet.index(alignments[s][alignments_star[star_index]])] += 1
            if statePath[i] == "I":
                # contiguous I case
                star_index = int(currentState[-1])-1
                alignment_index = alignments_star[star_index]
                available = [x for x in intervals if x > alignment_index]
                for a in available:
                    if alignments[s][a] != "-":
                        emissionMatrix[states.index(currentState)][alphabet.index(alignments[s][a])] += 1
                        intervals.remove(a)
                        break
            transmissionMatrix[states.index(lastState)][states.index(currentState)] += 1
            currentCol += 1
            
        # deal with the end point
        endState = statePath[-1] + (str(currentCol) if statePath[-1] !="I" else str(currentCol-1))
        if statePath[-1] == "M":
            star_index = int(endState[-1])-1
            emissionMatrix[states.index(endState)][alphabet.index(alignments[s][alignments_star[star_index]])] += 1
        if statePath[-1] == "I":
            star_index = int(endState[-1])-1
            alignment_index = alignments_star[star_index]
            available = [x for x in intervals if x > alignment_index]
            for a in available:
                if alignments[s][a] != "-":
                    emissionMatrix[states.index(endState)][alphabet.index(alignments[s][a])] += 1
                    intervals.remove(a)
                    break
            
        transmissionMatrix[states.index(endState)][-1] += 1

    # now the tarnsmissionMatrix is the count matrix of occured transitions, convert to ratio
    # Calculate the sum of each row
    transmission = np.array(transmissionMatrix)
    emission = np.array(emissionMatrix)
    row_sums1 = transmission.sum(axis=1, keepdims=True)
    row_sums2 = emission.sum(axis=1,keepdims=True)
    # Normalize each row by its sum
    transmissionProb = np.where(row_sums1 != 0, transmission / row_sums1, 0)
    emissionProb = np.where(row_sums2 != 0, emission / row_sums2, 0)

    dfTransimission = pd.DataFrame(transmissionProb, index=states, columns=states)
    dfEmission = pd.DataFrame(emissionProb, index=states, columns=alphabet)

    file_path1 = "transition_result1.txt"
    file_path2 = "emission_result1.txt"
    with open(file_path1, 'w') as file:
        header = ' \t' + '\t'.join(states) + '\n'
        file.write(header)
        for i, row in enumerate(transmissionProb):
            # Convert each element to string and join them with spaces
            line = states[i] + '\t' + '\t'.join(map(str, row))
            file.write(line + '\n')
    with open(file_path2, 'w') as file:
        header = ' \t' + '\t'.join(alphabet) + '\n'
        file.write(header)
        for i, row in enumerate(emissionProb):
            # Convert each element to string and join them with spaces
            line = states[i] + '\t' + '\t'.join(map(str, row))
            file.write(line + '\n')
    return dfTransimission, dfEmission

           
profileHMM_para(threshold, alphabet, alignments)
# print("--------")
# print(profileHMM_para(threshold, alphabet, alignments)[1])
# warnings.filterwarnings('default', category=RuntimeWarning)

# 4. Profile HMM + pseudocount
# transition matrix is state*state (only cells corresponding to edges will have a pseudocount added)
# emission matrix is state*alphabet (all cells will have a pseudocount added?)
# this function is only designed for transition matrix, since we will initialize a matrix with pseudocount for emission matrix
def add_pseudocount(matrix, pseudocount):
    dimension = len(matrix)
    # no need to consider matrix[-1] which is E
    # dimension[0] and dimension[1] has the same grey grid
    for i in [0,1]:
        colIndex = [1,2,3] # this is I0, M1, D1
        for n in colIndex:
            matrix[i][n] += pseudocount
    # start from i = 2, that is [2, 5]: M1, D1, I1
    i = 2
    while i < dimension - 1:
        colEnd = min(i+5,dimension)
        for r in range(i,i+3):
            for c in range(i+2,colEnd):
                matrix[r][c] += pseudocount
        i += 3
    
    row_sums = matrix.sum(axis=1, keepdims=True)
    normalized = np.where(row_sums != 0, matrix / row_sums, 0)
    
    return normalized


def profileHMM_pseudocount_para(threshold, alphabet, alignments,pseudocount):
    # find the alignment_star accoridng to threshold
    align_size = len(alignments)
    string_size = len(alignments[0])
    # alignment_star = alignments.copy()
    columns_to_remove = set()
    for i in range(string_size):
        fraction_count = 0
        for j in range(align_size):
            if alignments[j][i] == '-':
                fraction_count += 1
        ratio = fraction_count/align_size
        
        if ratio >= threshold:
            columns_to_remove.add(i)

    # alignment_star stores the columns index that will be considered for the profile matrix.
    col_index = [i for i in range(len(alignments[0]))]
    alignments_star = [item for item in col_index if item not in columns_to_remove]
    
    # seed alignment defines the length of insertion states, match states, and deletion states 
    # generate and store all possible states nodes
    states = ["S", "I0"]
    for i in range(1, len(alignments_star)+1):
        states.append("M"+str(i))
        states.append("D"+str(i))
        states.append("I"+str(i))
    states.append("E")

    dimension = len(states)
    # build a transmissionMatrix, it is initialized as a count matrix, and will calculate into transmission probability later
    transmissionMatrix = [[0 for _ in range(dimension)]for _ in range(dimension)]
    emissionMatrix = [[0 for _ in range(len(alphabet))]for _ in range(dimension)]
   

    # traverse every string in alignments
    all_statePath = []
    for i in range(len(alignments)):
        statePath = []
        for j in range(len(alignments[i])):
            # insertion case
            if j in columns_to_remove:
                if alignments[i][j] != "-":
                    statePath.append("I")
                # here i should be j-1 state how to know?
                #transmissionMatrix[i][insertion] += 1
            # match case
            else:
                if alignments[i][j] != "-":
                    statePath.append("M")
                # deletion case
                elif alignments[i][j] == "-":
                    statePath.append("D")
        all_statePath.append(statePath)
        
    for s in range(len(all_statePath)):
        # deal with the start point
        currentCol = 1
        statePath = all_statePath[s]
        intervals = list(columns_to_remove.copy())
        if statePath[0] == "M":
            transmissionMatrix[0][states.index("M1")] += 1
            emissionMatrix[states.index("M1")][alphabet.index(alignments[s][alignments_star[0]])] += 1
        elif statePath[0] == "D":
            transmissionMatrix[0][states.index("D1")] += 1
        else:
            transmissionMatrix[0][states.index("I0")] += 1
            if alignments[s][intervals[0]] != "-":
                emissionMatrix[states.index("I0")][alphabet.index(alignments[s][intervals[0]])] += 1
                intervals.remove(intervals[0])
        
        
        for i in range(1, len(statePath)):
            # dealing with the start point is an example of this operation
            lastState = statePath[i-1]+ (str(currentCol) if statePath[i-1]!="I" else str(currentCol-1))
            currentCol = int(lastState[-1])
            currentState = statePath[i]+ (str(currentCol+1) if statePath[i]!="I" else str(currentCol))
            
            if statePath[i] == "M":
                star_index = int(currentState[-1])-1
                emissionMatrix[states.index(currentState)][alphabet.index(alignments[s][alignments_star[star_index]])] += 1
            if statePath[i] == "I":
                # contiguous I case
                star_index = int(currentState[-1])-1
                alignment_index = alignments_star[star_index]
                available = [x for x in intervals if x > alignment_index]
                for a in available:
                    if alignments[s][a] != "-":
                        emissionMatrix[states.index(currentState)][alphabet.index(alignments[s][a])] += 1
                        intervals.remove(a)
                        break
            transmissionMatrix[states.index(lastState)][states.index(currentState)] += 1
            currentCol += 1
            
        # deal with the end point
        endState = statePath[-1] + (str(currentCol) if statePath[-1] !="I" else str(currentCol-1))
        if statePath[-1] == "M":
            star_index = int(endState[-1])-1
            emissionMatrix[states.index(endState)][alphabet.index(alignments[s][alignments_star[star_index]])] += 1
        if statePath[-1] == "I":
            star_index = int(endState[-1])-1
            alignment_index = alignments_star[star_index]
            available = [x for x in intervals if x > alignment_index]
            for a in available:
                if alignments[s][a] != "-":
                    emissionMatrix[states.index(endState)][alphabet.index(alignments[s][a])] += 1
                    intervals.remove(a)
                    break
            
        transmissionMatrix[states.index(endState)][-1] += 1

    # now the tarnsmissionMatrix is the count matrix of occured transitions, convert to ratio
    # Calculate the sum of each row
    transmission = np.array(transmissionMatrix)
    emission = np.array(emissionMatrix)
    row_sums1 = transmission.sum(axis=1, keepdims=True)
    row_sums2 = emission.sum(axis=1,keepdims=True)
    # Normalize each row by its sum
    transmissionProb = np.where(row_sums1 != 0, transmission / row_sums1, 0)
    transmissionProb = add_pseudocount(transmissionProb,pseudocount)

    emissionProb = np.where(row_sums2 != 0, emission / row_sums2, 0)
    Signal = [i for i in range(dimension) if states[i][0] in ["I","M"]]
    for row in Signal:
        for i in range(len(alphabet)):
            emissionProb[row][i] += pseudocount
    row_sums_new = emissionProb.sum(axis=1,keepdims=True)
    emissionProbability = np.where(row_sums_new != 0, emissionProb / row_sums_new, 0)

    dfTransimission = pd.DataFrame(transmissionProb, index=states, columns=states)
    dfEmission = pd.DataFrame(emissionProbability, index=states, columns=alphabet)

    file_path1 = "transition_result_pseudo.txt"
    file_path2 = "emission_result_pseudo.txt"
    with open(file_path1, 'w') as file:
        header = ' \t' + '\t'.join(states) + '\n'
        file.write(header)
        for i, row in enumerate(transmissionProb):
            # Convert each element to string and join them with spaces
            line = states[i] + '\t' + '\t'.join(map(str, row))
            file.write(line + '\n')
    with open(file_path2, 'w') as file:
        header = ' \t' + '\t'.join(alphabet) + '\n'
        file.write(header)
        for i, row in enumerate(emissionProbability):
            # Convert each element to string and join them with spaces
            line = states[i] + '\t' + '\t'.join(map(str, row))
            file.write(line + '\n')
    return dfTransimission, dfEmission

print(profileHMM_pseudocount_para(threshold,alphabet,alignments,pseudocount))